# MÉTRICA 3_2_07 — Congruencia entre la actividad laboral del declarante y la información del empleo/cargo/comisión
| Escenario                                                   | Condición                                                                      | Resultado        |
| ----------------------------------------------------------- | ------------------------------------------------------------------------------ | ---------------- |
| No existe actividad laboral                                 | Campo vacío o nulo                                                             | ⚪ **SIN_DATO**   |
| Actividad = SERVIDOR PÚBLICO pero sin datos de empleo       | `actividadLaboral` = público y `datosEmpleoCargoComision` vacío                | 🔴 **NO_CUMPLE** |
| Actividad ≠ SERVIDOR PÚBLICO pero sí hay datos de empleo    | Actividad privada/negocios/no trabaja y `datosEmpleoCargoComision` tiene datos | 🔴 **NO_CUMPLE** |
| Actividad = NO TRABAJA pero existen datos de cargo/comisión | Contradicción directa                                                          | 🔴 **NO_CUMPLE** |
| Actividad laboral coincide con los datos de empleo          | Coincide actividad con presencia/ausencia de `datosEmpleoCargoComision`        | 🟢 **CUMPLE**    |
| Información insuficiente                                    | Ambos campos vienen incompletos                                                | ⚪ **SIN_DATO**   |


In [ ]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "3_2_07_CONGRUENCIA_ACTIVIDAD_LABORAL"
BATCH_SIZE = 10000

MAPEO_LABORAL = {
    "SERVIDOR PUBLICO": "PUBLICO",
    "SECTOR PUBLICO": "PUBLICO",
    "PRIVADO": "PRIVADO",
    "SECTOR PRIVADO": "PRIVADO",
    "NEGOCIOS PROPIOS": "NEGOCIOS",
    "EMPRESARIAL": "NEGOCIOS",
    "NO TRABAJA": "SIN_EMPLEO",
    "DESEMPLEADO": "SIN_EMPLEO",
    "NO APLICA": "SIN_EMPLEO",
}

def normalizar_actividad(valor):
    if not valor:
        return None
    v = valor.strip().upper()
    return MAPEO_LABORAL.get(v, v)

def tiene_datos_empleo(datos):
    if not datos:
        return False
    if isinstance(datos, dict):
        return any(v not in (None, "", []) for v in datos.values())
    return True

def evaluar_metrica(doc):
    actividad_raw = doc.get("declaracion", {}) \
                          .get("situacionPatrimonial", {}) \
                          .get("datosGenerales", {}) \
                          .get("actividadLaboral", "")

    actividad = normalizar_actividad(actividad_raw)

    datos_empleo = doc.get("declaracion", {}) \
                      .get("situacionPatrimonial", {}) \
                      .get("datosEmpleoCargoComision", None)

    hay_empleo = tiene_datos_empleo(datos_empleo)

    if not actividad:
        return "SIN_DATO"

    if actividad == "PUBLICO" and not hay_empleo:
        return "NO_CUMPLE"

    if actividad in ("PRIVADO", "NEGOCIOS") and hay_empleo:
        return "NO_CUMPLE"

    if actividad == "SIN_EMPLEO" and hay_empleo:
        return "NO_CUMPLE"

    return "CUMPLE"

def procesar_metrica():
    client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
    db = client[DB_NAME]
    src = db[SOURCE_COLLECTION_NAME]
    tgt = db[METRICS_COLLECTION_NAME]

    total_docs = src.estimated_document_count()
    print(f"🚀 Procesando {METRIC_ID} con {total_docs:,} documentos...")

    skip = 0
    procesados = 0
    lote = 0

    try:
        while True:
            cursor = list(src.find(
                {},
                {
                    "_id": 1,
                    "declaracion.situacionPatrimonial.datosGenerales.actividadLaboral": 1,
                    "declaracion.situacionPatrimonial.datosEmpleoCargoComision": 1
                }
            ).skip(skip).limit(BATCH_SIZE))

            if not cursor:
                break

            operaciones = []
            for doc in cursor:
                try:
                    resultado = evaluar_metrica(doc)
                except:
                    resultado = "SIN_DATO"

                operaciones.append(UpdateOne(
                    {"_id": doc["_id"]},
                    {"$set": {METRIC_ID: resultado}}
                ))

            tgt.bulk_write(operaciones)
            procesados += len(operaciones)
            lote += 1

            print(f" 🔄 Lote {lote} → {procesados:,}/{total_docs:,} procesados")

            skip += BATCH_SIZE

        print("\n✅ Procesamiento completado.")

    except Exception:
        traceback.print_exc()
    finally:
        client.close()
        print("🔒 Conexión cerrada.")

if __name__ == "__main__":
    procesar_metrica()
